# LangChain是什么

LangChain 是用于构建大模型应用程序的开源框架，有 Python 和 JavaScript 两个不同版本的包。它由
模块化的组件构成，可单独使用也可链式组合实现端到端应用。
LangChain的组件包括：
* 提示(Prompts): 使模型执行操作的方式。
* 模型(Models)：大语言模型、对话模型，文本表示模型。目前包含多个模型的集成。
* 索引(Indexes): 获取数据的方式，可以与模型结合使用。
* 链(Chains): 端到端功能实现。
* 代理(Agents): 使用模型作为推理引擎

## 1.1 数据加载器

用户个人数据可以以多种形式呈现：PDF 文档、视频、网页等。基于 LangChain 提供给 LLM 访问用户
个人数据的能力，首先要加载并处理用户的多样化、非结构化个人数据。LangChain提供了各种类型的数据加载器，
如：PDFDataLoader、TextLoader、CSVLoader、JSONLoader、UnstructuredFileLoader、UnstructuredURLLoader等。


In [5]:
#pdf 加载器：

#!pip install pypdf

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("test/1.pdf")

pages = loader.load()

print(type(pages))  # <class 'list'>

print(len(pages))



#在 page 变量中，每一个元素都代表一个文档，它们的数据类型是 langchain.schema.Document
page = pages[0]
print(type(page))  # <class 'langchain.schema.document.Document'>
# page 的数据结构如下：
#      page_content ：包含该文档页面的内容
#      metadata ：包含该文档的元数据 ,比如：{'source': 'test/1.pdf/第一回：Matplotlib初相识.pdf', 'page': 0}
print(page.page_content)
print(page.metadata)

<class 'list'>
4
<class 'langchain_core.documents.base.Document'>
algorithm-visualizer.org
算法 可视化网站，点击 play 就可以看到每一行代码的执行过程对应的动画。有 java ，  c++ ，  javascript
等语言。 
learngitbranching.js.org/
一个学习 git 的可视化网站，帮助你像玩游戏一样掌握 git 的各种命令。 git 的重要性不需要多说，入门通 
过这个网站学习可能会更加容易消化。 
 
 
 
算法 可视化网站 
git 可是化网站 
https://www.liaoxuefeng.com/wiki/0013739516305929606dd18361248578c67b8067c8c017b000
{'producer': 'Skia/PDF m100', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) ynote-desktop/8.0.0 Chrome/100.0.4896.160 Electron/18.3.5 Safari/537.36', 'creationdate': '2025-06-23T06:51:13+00:00', 'moddate': '2025-06-23T06:51:13+00:00', 'source': 'test/1.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}


In [7]:
#短句分割
# 导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
chunk_size = 20 #设置块大小
chunk_overlap = 10 #设置块重叠大小
# 初始化递归字符文本分割器
r_splitter = RecursiveCharacterTextSplitter(
chunk_size=chunk_size,
chunk_overlap=chunk_overlap
) 
#初始化字符文本分割器
c_splitter = CharacterTextSplitter(
chunk_size=chunk_size,
chunk_overlap=chunk_overlap
)


text = "在AI的研究中，由于大模型规模非常大，模型参数很多，在大模型上跑完来验证参数好不好训练时间成本很高，所以一般会在小模型上做消融实验来验证哪些改进是有效的再去大模型上做实验。"

#测试文本
r_splitter.split_text(text)
print("*************")
c_splitter.split_text(text)


*************


['在AI的研究中，由于大模型规模非常大，模型参数很多，在大模型上跑完来验证参数好不好训练时间成本很高，所以一般会在小模型上做消融实验来验证哪些改进是有效的再去大模型上做实验。']

In [10]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=20)

text_splitter.split_text(text)

['在AI的研究中，由于大模型规模非常大，模型参数很多，在大模型上跑完来验证参数好不好训练时间成',
 '�数好不好训练时间成本很高，所以一般会在小模型上做消融实验来验证哪些改进是有效的再去大模型上做',
 '有效的再去大模型上做实验。']

## 1.2 数据分片（为什么要进行文档分割）

* 模型大小和内存限制：GPT 模型，特别是大型版本如 GPT-3 或 GPT-4 ，具有数十亿甚至上百亿的
参数。为了在一次前向传播中处理这么多的参数，需要大量的计算能力和内存。但是，大多数硬件
设备（例如 GPU 或 TPU ）有内存限制。文档分割使模型能够在这些限制内工作。
*计算效率：处理更长的文本序列需要更多的计算资源。通过将长文档分割成更小的块，可以更高效
地进行计算。
* 序列长度限制：GPT 模型有一个固定的最大序列长度，例如2048个 token 。这意味着模型一次只
能处理这么多 token 。对于超过这个长度的文档，需要进行分割才能被模型处理。
* 更好的泛化：通过在多个文档块上进行训练，模型可以更好地学习和泛化到各种不同的文本样式和
结构。
* 数据增强：分割文档可以为训练数据提供更多的样本。例如，一个长文档可以被分割成多个部分，
并分别作为单独的训练样本。

## 1.3 数据向量化

在机器学习和自然语言处理（NLP）中， Embeddings （嵌入）是一种将类别数据，如单词、句子或者整
个文档，转化为实数向量的技术。这些实数向量可以被计算机更好地理解和处理。嵌入背后的主要想法
是，相似或相关的对象在嵌入空间中的距离应该很近。

我们可以使用词嵌入（word embeddings）来表示文本数据。在词嵌入中，每个单词被转换
为一个向量，这个向量捕获了这个单词的语义信息。

## 1.4 数据检索

在构建检索增强生成 (RAG) 系统时，信息检索是核心环节。检索模块负责对用户查询进行分析，从知识
库中快速定位相关文档或段落，为后续的语言生成提供信息支持。检索是指根据用户的问题去向量数据
库中搜索与问题相关的文档内容，当我们访问和查询向量数据库时可能会运用到如下几种技术：
* 基本语义相似度(Basic semantic similarity)
* 最大边际相关性(Maximum marginal relevance，MMR)
* 过滤元数据
* LLM辅助检索

In [11]:
'''
    如何使用 RAG 丰富提示词？
	step 1：在RAG向量数据库中查找原始问题的答案
	docs = chatbot_st.query_from_doc(user_input, 3)
 
	step 2：使用Reranker对答案进行排序
    if chatbot_st.use_reranker:
        docs = chatbot_st.reranker.compress_documents(documents=docs, query=user_input)
        logging.info("using reranker now!!!!")

	step 3：将相似度最高的向量对应的文本作为提示词的组成部分，拼接成新的提示词。
    refer = "\n".join([x.page_content.replace("\n", '\t') for x in docs])
    PROMPT = """{}。\n请根据下面的参考文档回答上述问题。\n{}\n"""
    prompt = PROMPT.format(user_input, refer)
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
                                model=chatbot_st.llm,
                                messages=messages,
                                stream=True)
'''


'\n    如何使用 RAG 丰富提示词？\n\tstep 1：在RAG向量数据库中查找原始问题的答案\n\tdocs = chatbot_st.query_from_doc(user_input, 3)\n \n\tstep 2：使用Reranker对答案进行排序\n    if chatbot_st.use_reranker:\n        docs = chatbot_st.reranker.compress_documents(documents=docs, query=user_input)\n        logging.info("using reranker now!!!!")\n\n\tstep 3：将相似度最高的向量对应的文本作为提示词的组成部分，拼接成新的提示词。\n    refer = "\n".join([x.page_content.replace("\n", \'\t\') for x in docs])\n    PROMPT = """{}。\n请根据下面的参考文档回答上述问题。\n{}\n"""\n    prompt = PROMPT.format(user_input, refer)\n    messages = [{"role": "user", "content": prompt}]\n    response = client.chat.completions.create(\n                                model=chatbot_st.llm,\n                                messages=messages,\n                                stream=True)\n'

### 1.5 向量化数据计算相似性：

Faiss 提供了多种索引类型，支持不同的距离计算方式：

|距离度量 |Faiss 索引类型	|适用场景|
|:-:|:-:|:-:|
|L2（欧氏距离）	|IndexFlatL2	|图像检索、几何相似性|
内积（IP）	|IndexFlatIP	|余弦相似度（需先归一化向量）|
|余弦相似度	|IndexFlatIP + 向量归一化	|文本相似度（BERT 等）|
|Jaccard	|IndexBinaryFlat（二进制向量）	|集合相似度|
|Hamming	|IndexBinaryFlat（二进制向量）	|汉明距离（如 SimHash）|


In [15]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.11.0-cp310-cp310-win_amd64.whl.metadata (5.0 kB)
Using cached faiss_cpu-1.11.0-cp310-cp310-win_amd64.whl (15.0 MB)


In [ ]:
import numpy as np
import faiss
import time

d = 64  # 向量维度
nb = 10000  # 数据库大小
nq = 5  # 查询数量
np.random.seed(int(time.time()))
xb = np.random.random((nb, d)).astype('float32')  # 数据库向量
xq = np.random.random((nq, d)).astype('float32')  # 查询向量

# 使用 L2 距离（欧氏距离）
index = faiss.IndexFlatL2(d)
index.add(xb)
k = 4
D, I = index.search(xq, k)

print("L2 距离（欧氏距离）搜索结果:")
print("最近邻索引:\n", I)
print("对应距离:\n", D)


In [ ]:
# 使用内积（IP）
index_ip = faiss.IndexFlatIP(d)
index_ip.add(xb)
D_ip, I_ip = index_ip.search(xq, k)

print("\n内积（IP）搜索结果:")
print("最近邻索引:\n", I_ip)
print("对应内积值:\n", D_ip)

In [20]:
# 归一化向量（L2 归一化）
xb_normalized = xb / np.linalg.norm(xb, axis=1, keepdims=True)
xq_normalized = xq / np.linalg.norm(xq, axis=1, keepdims=True)

# 使用内积（IP）计算余弦相似度
index_ip_normalized = faiss.IndexFlatIP(d)
index_ip_normalized.add(xb_normalized)
D_cosine, I_cosine = index_ip_normalized.search(xq_normalized, k)

print("\n余弦相似度搜索结果（归一化后）:")
print("最近邻索引:\n", I_cosine)
print("对应余弦相似度（内积）:\n", D_cosine)


余弦相似度搜索结果（归一化后）:
最近邻索引:
 [[9284 1602 8146 8795]
 [3505 4938 9438 1607]
 [2471  702 3017 2504]
 [4459 6301 8219 3446]
 [2436 3954 7313 2207]]
对应余弦相似度（内积）:
 [[0.89410603 0.89023954 0.8811041  0.87891626]
 [0.87494147 0.86103094 0.8610229  0.8599931 ]
 [0.8652553  0.86274815 0.8623605  0.8580036 ]
 [0.8737376  0.86424994 0.8635894  0.862582  ]
 [0.8566244  0.837507   0.8359357  0.83164954]]
